In [1]:
import pygplates
import glob, re
import numpy as np
import matplotlib.pyplot as plt
import paleogeography as pg

%matplotlib inline

def determine(moving_plate_id,fixed_plate_id,time_from,time_to,rotation_model):
    angle = rotation_model.get_rotation(time_to,moving_plate_id,time_from,fixed_plate_id)
    return angle.represents_identity_rotation()


def split_plate_id_list(plate_id_list,time_from,time_to,rotation_model):
    
    fixed_plate_id = plate_id_list[0]
    
    plate_id_list_copy_True = list(plate_id_list)
    plate_id_list_copy_False = list(plate_id_list)
    
    plate_id_list_copy_True[:] = [tup for tup in plate_id_list_copy_True if determine(tup,fixed_plate_id,time_from,time_to,rotation_model)]
    plate_id_list_copy_False[:] = [tup for tup in plate_id_list_copy_False if not determine(tup,fixed_plate_id,time_from,time_to,rotation_model)]

    return plate_id_list_copy_True,plate_id_list_copy_False


def get_plate_motion_groups(plate_id_list,rotation_model,time,delta_time=1.):
    
    time_to = time
    time_from = time+delta_time
    
    plate_group_lists = []
    list_moving = list(plate_id_list)
    
    while len(list_moving)>0:
        list_fixed,list_moving = split_plate_id_list(list_moving,time_from,time_to,rotation_model)
        plate_group_lists.append(list_fixed)
        
    return plate_group_lists


def group_features_by_motion(features,rotation_model,time,delta_time=1,plot=False):

    rps = []
    pygplates.reconstruct(pg_features,rotation_model,rps,time)

    pids = []
    for rp in rps:
        pids.append(rp.get_feature().get_reconstruction_plate_id())

    pg_pids = list(set(pids))
    
    plate_group_lists = get_plate_motion_groups(pg_pids,rotation_model,time,delta_time)

    #print 'Number of groups is %d' % len(plate_group_lists)
    #for plate_group in plate_group_lists:
    #    print plate_group

    if plot:
        cmap = plt.get_cmap('hsv')
        colors = cmap(np.linspace(0, 1, len(plate_group_lists)))

        plt.figure(figsize=(20,9))
        for rp in rps:
            for (plate_group,color) in zip(plate_group_lists,colors):
                if rp.get_feature().get_reconstruction_plate_id() in plate_group:
                    plt.plot(rp.get_reconstructed_geometry().to_lat_lon_array()[:,1],
                             rp.get_reconstructed_geometry().to_lat_lon_array()[:,0],color=color)
        plt.show()   
                    
    return plate_group_lists




#plate_group_lists = get_plate_motion_groups(plate_id_list,rotation_model,time)
#
#print 'Number of groups is %d' % len(plate_group_lists)
#for plate_group in plate_group_lists:
#    print plate_group



ImportError: No module named paleogeography

In [ ]:
rotation_model = pygplates.RotationModel('/Applications/GPlates-2.0.0/SampleData/FeatureCollections/Rotations/Matthews_etal_GPC_2016_410-0Ma_GK07.rot')

env_list = ['lm','m','sm','i']

basedir = '/Users/Simon/Data/GPlates/PublishedModels/WCao_Paleogeography/Supplements/PresentDay_Paleogeog_Matthews2016_410-2Ma'

tmp = glob.glob('%s/*/' % basedir)

time_list = []
for tm in tmp:
    time_list.append(float(re.findall(r'\d+Ma+',tm)[1][:-2]))
#time_list = [396.,368.,348.,328.,302.,287.,277.,255.,232.,218.,195.]

time_list.sort()

for time in time_list:
    pg_dir = '%s/PresentDay_Paleogeog_Matthews2016_%dMa/' % (basedir,time)
    pg_features = pg.load_paleogeagraphy(pg_dir,env_list)
    pids = group_features_by_motion(pg_features,rotation_model,time,plot=True)
    print 'Number of groups for time %0.2f is %d' % (time,len(pids))




In [ ]:
for time_to,time_from in zip(time_list[:-1],time_list[1:]):
    pg_dir = '%s/PresentDay_Paleogeog_Matthews2016_%dMa/' % (basedir,time)
    pg_features = pg.load_paleogeagraphy(pg_dir,env_list)
    pids = group_features_by_motion(pg_features,rotation_model,time_to,time_from,plot=False)
    #print 'Number of groups for time %0.2f is %d' % (time,len(pids))
    
    rps = []
    pygplates.reconstruct(pg_features,rotation_model,rps,time_to)

    
    for rp in rps:
        group_features = []
        for plate_group in plate_group_lists:
            if rp.get_feature().get_reconstruction_plate_id() in plate_group:
                group_features.append(rp)
                    
    
    